In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

# --- 1. Chargement et Préparation des données ---
print("Chargement des données...")
df = pd.read_csv('./data/absences_data.csv') # Assure-toi que le chemin est correct

# --- 2. Encodage des variables catégorielles ---
print("Encodage des variables...")
# La seule variable catégorielle est notre cible 'decision_absence'
encoder_decision_absence = LabelEncoder()

# On encode la colonne cible et on la stocke dans une nouvelle colonne
df['decision_absence_encoded'] = encoder_decision_absence.fit_transform(df['decision_absence'])

# Afficher le mapping pour comprendre
print("Mapping de la décision :")
for i, classe in enumerate(encoder_decision_absence.classes_):
    print(f"{classe} -> {i}")

# --- 3. Définition des Features (X) et de la Cible (y) ---
features = ['duree_absence_jours', 'solde_conges_jours', 'nb_absences_injustifiees_annee', 'est_adjacent_weekend_ferie', 'charge_equipe']
target = 'decision_absence_encoded'

X = df[features]
y = df[target]

# --- 4. Division des données en ensembles d'entraînement et de test ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 5. Création et Entraînement du modèle LightGBM ---
print("\nEntraînement du modèle LightGBM...")
lgbm_classifier = lgb.LGBMClassifier(objective='multiclass', num_class=3, random_state=42)
lgbm_classifier.fit(X_train, y_train)

# --- 6. Évaluation du modèle ---
print("\nÉvaluation du modèle...")
y_pred = lgbm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle sur l'ensemble de test : {accuracy:.2f}")
print("\nRapport de classification :")
# On utilise inverse_transform pour afficher les vrais noms des classes dans le rapport
print(classification_report(y_test, y_pred, target_names=encoder_decision_absence.classes_))

# --- 7. Sauvegarde du modèle et de l'encodeur ---
print("\nSauvegarde du modèle et de l'encodeur...")
output_dir = './models'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Sauvegarde du modèle
model_path = os.path.join(output_dir, 'model_absence_injustifiee.pkl')
joblib.dump(lgbm_classifier, model_path)
print(f"Modèle sauvegardé dans : {model_path}")

# Sauvegarde de l'encodeur de décision
encoder_path = os.path.join(output_dir, 'encoder_decision_absence.pkl')
joblib.dump(encoder_decision_absence, encoder_path)
print(f"Encodeur sauvegardé dans : {encoder_path}")

print("\nProcessus terminé avec succès.")

Chargement des données...
Encodage des variables...
Mapping de la décision :
AVERTISSEMENT -> 0
DECOMPTE_SOLDE -> 1
JUSTIFICATIF_REQUIS -> 2

Entraînement du modèle LightGBM...
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 12, number of used features: 0
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] 

C:\Users\hjirt\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\hjirt\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\hjirt\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
